In [1]:
import sys
sys.path.append("..")
import argparse
import numpy as np
import dgl
from dgl import DGLGraph
import torch
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from collections import Counter
import pickle
import h5py
import random
import glob2
import seaborn as sns
import scanpy.api as sc
import train
import models
from sklearn.metrics import (accuracy_score, adjusted_rand_score,
                             calinski_harabasz_score,
                             normalized_mutual_info_score, silhouette_score)
%load_ext autoreload
%autoreload 2

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
device = train.get_device()

Using backend: pytorch
/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Detected an old version of PyTorch. Suggest using torch>=1.5.0 for the best experience.
  return warnings.warn(message, category=category, stacklevel=1)
/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
category = "real_data"

epochs = 10
batch_size = 128 
pca_size = 50
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
# print(files)


In [3]:
files = ["Quake_10x_Bladder", "Quake_Smart-seq2_Trachea"]

In [4]:
results = pd.DataFrame()
model_name = "GraphConv"
normalize_weights = "log_per_cell"
node_features = "scale"
same_edge_values = False
edge_norm = True
hidden_relu = False
hidden_bn = False
n_layers = 1
hidden_dim = 200
hidden = [300]
nb_genes = 3000
activation = F.relu
run = 0
df = pd.DataFrame()
for dataset in files:
    print(f">> {dataset}")

    data_mat = h5py.File(f"{path}/real_data/{dataset}.h5", "r")

    Y = np.array(data_mat['Y'])
    X = np.array(data_mat['X'])
    n_clusters = len(np.unique(Y))

    genes_idx, cells_idx = train.filter_data(X, highly_genes=nb_genes)
    X = X[cells_idx][:, genes_idx]
    Y = Y[cells_idx]

    t0 = time.time()
    graph = train.make_graph(
        X,
        Y,
        dense_dim=pca_size,
        node_features=node_features,
        normalize_weights=normalize_weights,
    )

    labels = graph.ndata["label"]
    train_ids = np.where(labels != -1)[0]

    sampler = dgl.dataloading.MultiLayerFullNeighborSampler(n_layers)

    dataloader = dgl.dataloading.NodeDataLoader(
        graph,
        train_ids,
        sampler,
        batch_size=batch_size,
        shuffle=True,
        drop_last=False,
        num_workers=1,
    )
    print(
        f"INPUT: {model_name}  {hidden_dim}, {hidden}, {same_edge_values}, {edge_norm}"
    )
    t1 = time.time()
    for resolution in [0.05, 0.1, 0.2, 0.5,0.9, 1, 1.2]:
        t_start = time.time()
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)

        model = models.GCNAE(
            in_feats=pca_size,
            n_hidden=hidden_dim,
            n_layers=n_layers,
            activation=activation,
            dropout=0.1,
            hidden=hidden,
            hidden_relu=hidden_relu,
            hidden_bn=hidden_bn,
        ).to(device)

        optim = torch.optim.Adam(model.parameters(), lr=1e-5)

        scores = train.train(
            model,
            optim,
            epochs,
            dataloader,
            n_clusters,
            plot=False,
            save = False,
            cluster=["Leiden"],
            cluster_params={"Leiden": {
                "resolution": resolution

            }})
        print(f"resolution : {resolution}")
        scores["dataset"] = dataset
        scores["resolution"] = resolution
        scores["nb_pred_clusters"] = np.unique(scores["leiden_pred"]).shape[0]
        scores["true_nb_clusters"] = np.unique(Y).shape[0]
        results = results.append(scores, ignore_index=True)

        results.to_pickle(
            f"../output/pickle_results/{category}/{category}_leiden.pkl")
        print("Done")
        display(results.tail())
#         break
#     break

results.mean()  #

>> Quake_10x_Bladder


/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


INPUT: GraphConv  200, [300], False, True



ARI None, None
resolution : 0.05
Done


,ae_end,aris_kmeans,dataset,leiden_ari,leiden_cal,leiden_nmi,leiden_pred,leiden_sil,leiden_time,nb_pred_clusters,resolution,true_nb_clusters
0,1.629019e+09,[],Quake_10x_Bladder,0.8973,3335.233551,0.8657,"[0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, ...",0.668833,17.748664,2.0,0.05,4.0


...


ARI None, None
resolution : 1
Done


,ae_end,aris_kmeans,dataset,leiden_ari,leiden_cal,leiden_nmi,leiden_pred,leiden_sil,leiden_time,nb_pred_clusters,resolution,true_nb_clusters
8,1.629019e+09,[],Quake_Smart-seq2_Trachea,0.5628,300.963488,0.6731,"[5, 3, 1, 3, 3, 0, 1, 3, 4, 0, 0, 1, 0, 0, 0, ...",0.384986,2.514596,6.0,0.1,4.0
9,1.629019e+09,[],Quake_Smart-seq2_Trachea,0.5436,364.806402,0.6831,"[6, 4, 2, 4, 4, 0, 2, 4, 3, 0, 0, 2, 0, 0, 0, ...",0.439242,2.484028,8.0,0.2,4.0
10,1.629019e+09,[],Quake_Smart-seq2_Trachea,0.3123,339.887894,0.5912,"[9, 5, 2, 5, 5, 0, 2, 5, 8, 1, 0, 2, 0, 0, 0, ...",0.336664,2.498735,11.0,0.5,4.0
11,1.629019e+09,[],Quake_Smart-seq2_Trachea,0.3109,339.548425,0.5900,"[9, 5, 2, 5, 5, 0, 2, 5, 8, 1, 0, 2, 0, 0, 0, ...",0.336427,2.503169,11.0,0.9,4.0
12,1.629019e+09,[],Quake_Smart-seq2_Trachea,0.2753,312.906594,0.5717,"[10, 4, 2, 4, 4, 0, 2, 4, 9, 1, 0, 2, 0, 0, 0,...",0.293150,2.796522,12.0,1.0,4.0



ARI None, None
resolution : 1.2
Done


,ae_end,aris_kmeans,dataset,leiden_ari,leiden_cal,leiden_nmi,leiden_pred,leiden_sil,leiden_time,nb_pred_clusters,resolution,true_nb_clusters
9,1.629019e+09,[],Quake_Smart-seq2_Trachea,0.5436,364.806402,0.6831,"[6, 4, 2, 4, 4, 0, 2, 4, 3, 0, 0, 2, 0, 0, 0, ...",0.439242,2.484028,8.0,0.2,4.0
10,1.629019e+09,[],Quake_Smart-seq2_Trachea,0.3123,339.887894,0.5912,"[9, 5, 2, 5, 5, 0, 2, 5, 8, 1, 0, 2, 0, 0, 0, ...",0.336664,2.498735,11.0,0.5,4.0
11,1.629019e+09,[],Quake_Smart-seq2_Trachea,0.3109,339.548425,0.5900,"[9, 5, 2, 5, 5, 0, 2, 5, 8, 1, 0, 2, 0, 0, 0, ...",0.336427,2.503169,11.0,0.9,4.0
12,1.629019e+09,[],Quake_Smart-seq2_Trachea,0.2753,312.906594,0.5717,"[10, 4, 2, 4, 4, 0, 2, 4, 9, 1, 0, 2, 0, 0, 0,...",0.293150,2.796522,12.0,1.0,4.0
13,1.629019e+09,[],Quake_Smart-seq2_Trachea,0.2217,291.694076,0.5514,"[11, 5, 2, 5, 5, 1, 2, 5, 10, 0, 4, 2, 4, 1, 4...",0.304671,2.854597,13.0,1.2,4.0


ae_end              1.629019e+09
leiden_ari          4.682214e-01
leiden_cal          1.402949e+03
leiden_nmi          6.600143e-01
leiden_sil          3.856226e-01
leiden_time         4.660209e+00
nb_pred_clusters    7.928571e+00
resolution          5.642857e-01
true_nb_clusters    4.000000e+00
dtype: float64

In [5]:
results[["dataset", "true_nb_clusters",  "resolution" , "nb_pred_clusters", "leiden_ari", "leiden_nmi", "leiden_sil", "leiden_cal",]].round(2)

,dataset,true_nb_clusters,resolution,nb_pred_clusters,leiden_ari,leiden_nmi,leiden_sil,leiden_cal
0,Quake_10x_Bladder,4.0,0.05,2.0,0.90,0.87,0.67,3335.23
1,Quake_10x_Bladder,4.0,0.10,3.0,0.72,0.75,0.59,2249.81
2,Quake_10x_Bladder,4.0,0.20,4.0,0.74,0.79,0.64,3311.76
3,Quake_10x_Bladder,4.0,0.50,6.0,0.53,0.73,0.39,3091.11
4,Quake_10x_Bladder,4.0,0.90,10.0,0.30,0.59,0.21,1843.53
5,Quake_10x_Bladder,4.0,1.00,11.0,0.25,0.56,0.20,1707.38
6,Quake_10x_Bladder,4.0,1.20,11.0,0.25,0.56,0.19,1728.37
7,Quake_Smart-seq2_Trachea,4.0,0.05,3.0,0.64,0.74,0.42,424.29
8,Quake_Smart-seq2_Trachea,4.0,0.10,6.0,0.56,0.67,0.38,300.96
9,Quake_Smart-seq2_Trachea,4.0,0.20,8.0,0.54,0.68,0.44,364.81
